In [ ]:
!pip install seaborn

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

# Limpeza de Dados

Dados do *mundo real* raramente são limpos: muitas vezes, para chegar na informação real precisamos penar e sofrer! A limpeza de dados (**data scrubbing** em inglês) é uma habilidade fundamental para qualquer pessoa trabalhando com dados (seja um cientista de dados, um analista de BI ou um engenheiro de dados!). Hoje vamos aprender algumas das técnicas que podemos utilizar para problemas gerais que encontramos em dados.

Vamos começar carregando nossos dados: utilizaremos hoje uma tabela semelhante à `tb_veic`: mas essa nova tabela contém mais informações! Infelizmente, muitas dessas informações estão com problemas, que tentaremos corrigir ao longo da aula hoje.

In [ ]:
tb_veic_messy = pd.read_csv("data/vehicles_messy.csv", dtype={"year": "object"})


In [ ]:
tb_veic_messy.head()

Como nosso DataFrame é muito longo (muitas colunas) não conseguimos vê-lo completamente através do método `.head()`. Para visualizar todas as colunas precisamos utilizar uma opção da biblioteca Pandas:

In [ ]:
pd.set_option('display.max_columns', 500)
tb_veic_messy.head()

## Data types

Uma das primeiras etapas na limpeza de dados é garantir que as colunas tenham os tipos desejados: colunas numéricas devem ser ints e floats, colunas de data, datetime, strings serem objects... Primeiro vamos ver algumas formas de determinar os tipos de nossas colunas.

### Checando tipos

In [ ]:
tb_veic_messy.info()

In [ ]:
tb_veic_messy.dtypes

In [ ]:
tb_veic_messy.dtypes.value_counts()

Além de acessar os tipos e conta-los, podemos utilizar o método `.select_dtypes()` para filtrar as colunas de um DataFrame de acordo com seu tipo:

In [ ]:
tb_veic_messy.select_dtypes(include = 'object')

In [ ]:
tb_veic_messy.select_dtypes(include = 'object').info()

### Convertendo tipos `.astype()`

Nas colunas acima podemos ver que a coluna `year` parece conter apenas números (o que faria sentido) mas está carregada com um `object` (o que não faz tanto sentido). Vamos investigar os dados dessa colunas para entendermos melhor o que ela contém:

In [ ]:
tb_veic_messy['year'].head(20)

In [ ]:
tb_veic_messy['year'].describe()

In [ ]:
tb_veic_messy['year'].unique()

Aparentemente a colunas contém apenas números. Neste caso, podemos utilizar o método `.astype()` para converte-la em um tipo numérico:

In [ ]:
tb_veic_messy['year_num'] = tb_veic_messy['year'].astype('int64')

In [ ]:
tb_veic_messy['year_num'].describe()

### Convertendo Tipos com `.map()`

Além do método `.astype()`, podemos utilizar o método `.map()` para converter o tipo de uma coluna. O método `.map()` **mapea uma função sobre os elementos de uma coluna**, ou seja, ele retorna uma nova coluna, do tamanho da coluna original, preenchido com o resultado da função sobre os elementos da coluna original (um pouco parecido com os List Comprehensions...).

Vamos utilizar este método para aplicar a função `int` aos elementos da coluna `year`:

In [ ]:
tb_veic_messy['year'].map(int)

Porque usar o `.map()`? Em muitas situações, o problema em nossos dados não será apenas de definição de tipo e algumas linhas da nossa coluna numérica terão strings misturados... Neste caso o método `.astype()` não funcionará:

In [ ]:
tb_veic_messy['year_num_errado'].unique()

In [ ]:
tb_veic_messy['year_num_errado'].astype('int64')

Passar a função `int` através do `.map()` também não funcionará!

In [ ]:
tb_veic_messy['year_num_errado'].map(int)

Mas o método `.map()` nos permite passar **QUALQUER FUNÇÃO** para sobre coluna! Vamos definir uma função que tenta converter um string em número mas que, ao invés de falhar com strings, retorna `np.nan`:

In [ ]:
def te_int(value):
    try:
        return int(value)
    except ValueError as e:
        return np.nan

tb_veic_messy['year_map'] = tb_veic_messy['year_num_errado'].map(te_int)
tb_veic_messy[['year_num_errado', 'year_map']].head()

Quando verificamos os valores da coluna `year_num_errado` vimos que alguns anos estavam com um caráctere extra (a letra `a`). Com a função que definimos acima, nossa nova coluna de inteiros não terá nenhum valor nessas linhas. Como podemos alterar nossa função para que ela limpe, não remova, as linhas com `strings` misturados?

In [ ]:
# EXERCICIO!

## Valores ausentes (missing values)

Um dos maiores problemas que vamos enfrentar quando trabalhamos com dados reais são os ***missing values***. Os ***missing values*** surgem de 3 formas:

1. Pela aplicação de funções em nosso código 
    * **Exemplo**: `.map()` que construímos acima;
1. Através de falhas na coleta/armazenagem de dados
    * **Exemplo**: em uma pesquisa física, alguns campos preenchidos estavam ilegíveis.
1. Para representar características reais da observação
    * **Exemplo**: em uma tabela de e-commerce, com o faturamento mês a mês de cada item, a coluna `desconto` pode aparecer como NA para representar produtos que não estavam promocionados

Antes de tratarmos NAs precisamos **ENTENDER** porque eles existem, se é possível (e desejável) tentar recuperar essa informação, se ela pode ser reconstruída a partir de outras colunas, etc...

Começaremos vendo como a biblioteca Pandas representa NAs no Python: o objeto `np.nan`

### `np.nan` - o número que não é número!

Porque ter um tipo específico para NAs? Vamos utilizar o exemplo do `.map()` acima para entedermos essa escolha dos desenvolvedores: quando tentamos aplicar a função `int` sobre nossa coluna uma linha com o valor `1985a` impediu a aplicação da função! Uma linha em 37842! Imagine se NAs fossem `strings`: não consegueríamos aplicar nenhuma função numérica sobre colunas com mesmo só um NA!

In [ ]:
type(np.nan)

In [ ]:
np.nan + 1

In [ ]:
tb_veic_messy['year_map'].head()

In [ ]:
tb_veic_messy.loc[0, 'year_map']

In [ ]:
tb_veic_messy.loc[0, 'year_map'] + 1

In [ ]:
tb_veic_messy['year_map'] + 1

Esse comportamento é um pouco diferente quando falamos de funções de agregação:

In [ ]:
tb_veic_messy['year_map'].head()

In [ ]:
tb_veic_messy['year_map'].mean()

Um cuidado que devemos tomar é que quando utilizamos relação de equivalência com `np.nan`:

In [ ]:
1 == 1

In [ ]:
None == None

In [ ]:
np.nan == np.nan

In [ ]:
tb_veic_messy[tb_veic_messy['year_map'] != np.nan].head()

### Os métodos `.isna()` e `.notna()`

Como vimos acima, para criar uma máscara representando valores `np.nan` não podemos utilizar uma relação de equivalência. Precisamos utilizar os métodos `.isna()` e `.notna()`!

In [ ]:
tb_veic_messy['displ'].head()

In [ ]:
tb_veic_messy['displ'].info()

In [ ]:
tb_veic_messy['displ'].isna()

In [ ]:
tb_veic_messy['displ'].notna()

In [ ]:
mask_na_displ = tb_veic_messy['displ'].isna()
tb_displ_na = tb_veic_messy[mask_na_displ]
tb_displ_na.info()

In [ ]:
sum(tb_veic_messy['displ'].isna())

### O método `.isna()` em DataFrames

Além de sua utilização em `Series`, o método `.isna()` pode ser utilizado em DataFrames: o resultado será um DataFrame de máscaras com a avaliação do método aplicada à cada coluna:

In [ ]:
tb_veic_messy.isna()

Agora podemos utilizar o método `.sum(axis = 1)` para contabilizar o número de NAs em cada linha!

In [ ]:
tb_veic_messy.isna().sum(axis = 1)

In [ ]:
tb_veic_messy['num_val_na'] = tb_veic_messy.isna().sum(axis = 1)

In [ ]:
tb_veic_messy['num_val_na'].describe()

Com essa nova coluna podemos construir filtros sobre a **completude** das linhas, ou seja, quantas colunas dela tem valores missing:

In [ ]:
mask_num_na = tb_veic_messy['num_val_na'] < 10
mask_num_na

In [ ]:
tb_veic_messy[mask_num_na].info()

### Calculando a % de Valores NA

Utilizando o que construímos até agora, podemos construir uma coluna com a informação da % de linhas que apresentam valores NAs.

In [ ]:
tb_veic_messy['per_val_na'] = tb_veic_messy.isna().sum(axis = 1)/tb_veic_messy.shape[1]

In [ ]:
tb_veic_messy['per_val_na'].describe()

## Removendo (***dropando***) colunas 
Como vimos antes, o método `.drop()` pode ser utilizado para remover linhas ou colunas. Quando queremos remover uma coluna de nosso tabela devemos usar um dois dois argumentos abaixo:

1. `axis=1` ou
1. `columns = ['nome_da_coluna1', 'nome_da_coluna2' ,...]`

In [ ]:
tb_veic_messy.drop('year', axis = 1).head()

In [ ]:
tb_veic_messy.drop(columns = ['year']).head()

### Removendo colunas condicionalmente

Uma técnica importante é a remoção de colunas através de condições. Por exemplo, nosso DataFrame pode ter colunas com um único valor. Vamos utilizar o método `.drop()` para remover essas colunas:

In [ ]:
tb_veic_messy.columns

In [ ]:
tb_veic_messy['cityA08'].unique()

In [ ]:
colunas_unique = []
for coluna in tb_veic_messy.columns:
    n_valores = len(tb_veic_messy[coluna].unique())
    if n_valores == 1:
        colunas_unique.append(coluna)
print(colunas_unique)

Vamos praticar nosso raciocinio funcional transformando o loop acima em um list comprehension!

In [ ]:
# EXERCICIO

Agora, com nossa lista de colunas, podemos utilizar o método `.drop()` para tratar nosso DataFrame:

In [ ]:
tb_veic_messy.drop(columns = colunas_unique).info()

## O método `.dropna()`

A maneira mais **direta**, *mas nem sempre correta*, de tratar NAs é removendo-os da nossa tabela. Como podemos saber quando remover NAs? Basta lembrar dos três motivos pelo qual eles aparecem:

1. Pela aplicação de funções em nosso código 
    * **Exemplo**: `.map()` que construímos acima;
1. Através de falhas na coleta/armazenagem de dados
    * **Exemplo**: em uma pesquisa física, alguns campos preenchidos estavam ilegíveis.
1. Para representar características reais da observação
    * **Exemplo**: em uma tabela de e-commerce, com o faturamento mês a mês de cada item, a coluna `desconto` pode aparecer como NA para representar produtos que não estavam promocionados

Na sua opinião, qual tipo de NA devemos manter e qual devemos excluir da base?

Uma vez determinado que vamos remover NAs temos duas formas de fazê-lo: excluindo colunas e excluindo linhas. Em geral excluímos colunas quando a quantidade de NAs é tão grande que seria impossível utilizar a informação da coluna para qualquer coisa. No caso das linhas, removemos linhas com NA **APENAS QUANDO AS COLUNAS DE NOSSA ANÁLISE FOREM IMPACTADAS!!**

Vamos ver como o método `.dropna()` pode nos ajudar aqui.

Primeiro em tudo:

In [ ]:
tb_veic_messy.dropna()

Agora nas linhas:

In [ ]:
tb_veic_messy.dropna(axis = 0)

Agora nas colunas:

In [ ]:
tb_veic_messy.dropna(axis = 1)

Independente do eixo (`axis`) de aplicação, podemos especificar o `thresh` - um patamar de NAs abaixo do qual removemos uma linha ou coluna. Se escolhermos o eixo das linhas, e colocarmos o `thresh = 60`, manteremos apenas as linhas com pelo menos 60 colunas preenchidas. Já no caso das colunas, o `thresh` será o mínimo de linhas não-NA que uma coluna precisar para continuar no DataFrame.

In [ ]:
tb_veic_messy.dropna(axis = 0, thresh = 80)

In [ ]:
tb_veic_messy.dropna(axis = 1, thresh = 5000)

In [ ]:
thresh_col = int(tb_veic_messy.shape[0] * 0.20)
thresh_col

In [ ]:
tb_veic_messy.dropna(axis = 1, thresh = thresh_col)

## O método `.fillna()`

Em algumas condições (quais?) é melhor substituir os ***missing values*** por algum outro valor. Para tanto vamos utilizar o método `.fillna()`.

In [ ]:
tb_veic_messy['trans_dscr'].isna().sum(axis = 0)

In [ ]:
tb_veic_messy['trans_dscr'].unique()

In [ ]:
tb_veic_messy['trans_dscr'].fillna('OUTROS')

In [ ]:
tb_veic_messy['trans_dscr_fna'] = tb_veic_messy['trans_dscr'].fillna('OUTROS')

In [ ]:
tb_veic_messy['trans_dscr_fna'].value_counts()

In [ ]:
tb_veic_messy\
    .select_dtypes(include = "number")\
    .isna().sum(axis = 0)\
    .sort_values(ascending=False)

In [ ]:
tb_veic_messy['cylinders'].unique()

Podemos utilizar a média de uma coluna numérica como substituto para valores NAs. Essa técnica chama-se imputação de valores nulos, e, no módulo 3 veremos formas menos rudimentares de utiliza-la!

In [ ]:
tb_veic_messy['cyl_impu'] = tb_veic_messy['cylinders'].fillna(tb_veic_messy['cylinders'].mean())
tb_veic_messy['cyl_impu'].unique()

In [ ]:
tb_veic_messy[['cyl_impu', 'cylinders']].describe()

## Duplicação de linhas

Em algumas situações, nossos DataFrames podem conter linas duplicadas. Para duplicações simples (onde todas as colunas são iguais) podemos utilizar o método `.duplicated()` para limpar as duplicatas.

In [ ]:
frutas = pd.DataFrame({
    'fruta':['laranja', 'laranja', 'mamão', 'laranja'],
    'cidade':['Rio de Janeiro', 'Atibaia', 'Campinas', 'Rio de Janeiro']
})

In [ ]:
frutas

In [ ]:
frutas.duplicated()

In [ ]:
frutas.loc[~frutas.duplicated(),]